# Download Data and Pre-Trained Model

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Colab'

In [2]:
# !wget https://s3.amazonaws.com/matroid-web/datasets/agegender_cleaned.tar.gz -P './data'
# !wget https://m-training.s3-us-west-2.amazonaws.com/dlchallenge/vgg_face_matconvnet.tar.gz -P './data'
# !tar -xvf ./data/agegender_cleaned.tar.gz 
# !tar -xvf ./data/vgg_face_matconvnet.tar.gz

In [3]:
# !pip install tensorflow==1.14.0
# !pip install keras==2.2.5
# !pip list | grep keras

In [4]:
import scipy.io
mat = scipy.io.loadmat('./vgg_face_matconvnet/data/vgg_face.mat', matlab_compatible=False, struct_as_record=False)

# Check Structure

In [5]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.models import load_model

In [6]:
net = mat['net'][0][0]
ref_model_layers = net.layers
ref_model_layers.shape

(1, 39)

In [7]:
ref_model_layers = net.layers[0]
for layer in ref_model_layers:
    print(layer[0][0].name)

['conv1_1']
['relu1_1']
['conv1_2']
['relu1_2']
['pool1']
['conv2_1']
['relu2_1']
['conv2_2']
['relu2_2']
['pool2']
['conv3_1']
['relu3_1']
['conv3_2']
['relu3_2']
['conv3_3']
['relu3_3']
['pool3']
['conv4_1']
['relu4_1']
['conv4_2']
['relu4_2']
['conv4_3']
['relu4_3']
['pool4']
['conv5_1']
['relu5_1']
['conv5_2']
['relu5_2']
['conv5_3']
['relu5_3']
['pool5']
['fc6']
['relu6']
['dropout6']
['fc7']
['relu7']
['dropout7']
['fc8']
['softmax']


In [8]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), name= 'conv1_1'))
model.add(Activation('relu', name='relu1_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), name= 'conv1_2'))
model.add(Activation('relu', name='relu1_2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool1'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), name= 'conv2_1'))
model.add(Activation('relu', name='relu2_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), name= 'conv2_2'))
model.add(Activation('relu', name='relu2_2'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool2'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_1'))
model.add(Activation('relu', name='relu3_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_2'))
model.add(Activation('relu', name='relu3_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), name= 'conv3_3'))
model.add(Activation('relu', name='relu3_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool3'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_1'))
model.add(Activation('relu', name='relu4_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_2'))
model.add(Activation('relu', name='relu4_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv4_3'))
model.add(Activation('relu', name='relu4_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool4'))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_1'))
model.add(Activation('relu', name='relu5_1'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_2'))
model.add(Activation('relu', name='relu5_2'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), name= 'conv5_3'))
model.add(Activation('relu', name='relu5_3'))
model.add(MaxPooling2D((2,2), strides=(2,2), name='pool5'))

model.add(Convolution2D(4096, (7, 7), name= 'fc6'))
model.add(Activation('relu', name='relu6'))
model.add(Dropout(0.5, name='dropout6'))
model.add(Convolution2D(4096, (1, 1), name= 'fc7'))
model.add(Activation('relu', name='relu7'))
model.add(Dropout(0.5, name='dropout7'))
model.add(Convolution2D(2622, (1, 1), name= 'fc8'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Activation('sigmoid', name= 'sigmoid'))

In [9]:
num_of_ref_model_layers = ref_model_layers.shape[0]
base_model_layer_names = [layer.name for layer in model.layers]

In [10]:
basemodel_layer_names = base_model_layer_names[:44]
model_layer_names = base_model_layer_names[44:]

In [11]:
for layer in model.layers:
    layer_name = layer.name
    try:
        print(layer_name,": ",layer.weights[0].shape)
    except:
        print("",end='')
        #print(layer_name)

conv1_1 :  (3, 3, 3, 64)
conv1_2 :  (3, 3, 64, 64)
conv2_1 :  (3, 3, 64, 128)
conv2_2 :  (3, 3, 128, 128)
conv3_1 :  (3, 3, 128, 256)
conv3_2 :  (3, 3, 256, 256)
conv3_3 :  (3, 3, 256, 256)
conv4_1 :  (3, 3, 256, 512)
conv4_2 :  (3, 3, 512, 512)
conv4_3 :  (3, 3, 512, 512)
conv5_1 :  (3, 3, 512, 512)
conv5_2 :  (3, 3, 512, 512)
conv5_3 :  (3, 3, 512, 512)
fc6 :  (7, 7, 512, 4096)
fc7 :  (1, 1, 4096, 4096)
fc8 :  (1, 1, 4096, 2622)


In [12]:
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    
    try:
        weights = ref_model_layers[i][0,0].weights[0,0]
        print(ref_model_layer,": ",weights.shape)
    except:
        #print(ref_model_layer)
        print("",end='')

conv1_1 :  (3, 3, 3, 64)
conv1_2 :  (3, 3, 64, 64)
conv2_1 :  (3, 3, 64, 128)
conv2_2 :  (3, 3, 128, 128)
conv3_1 :  (3, 3, 128, 256)
conv3_2 :  (3, 3, 256, 256)
conv3_3 :  (3, 3, 256, 256)
conv4_1 :  (3, 3, 256, 512)
conv4_2 :  (3, 3, 512, 512)
conv4_3 :  (3, 3, 512, 512)
conv5_1 :  (3, 3, 512, 512)
conv5_2 :  (3, 3, 512, 512)
conv5_3 :  (3, 3, 512, 512)
fc6 :  (7, 7, 512, 4096)
fc7 :  (1, 1, 4096, 4096)
fc8 :  (1, 1, 4096, 2622)


# Base Model

In [13]:
import warnings
warnings.filterwarnings('ignore')

base_model = Sequential()
base_model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
base_model.add(Convolution2D(64, (3, 3), name= 'conv1_1'))
base_model.add(Activation('relu', name='relu1_1'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(64, (3, 3), name= 'conv1_2'))
base_model.add(Activation('relu', name='relu1_2'))
base_model.add(MaxPooling2D((2,2), strides=(2,2), name='pool1'))

base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(128, (3, 3), name= 'conv2_1'))
base_model.add(Activation('relu', name='relu2_1'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(128, (3, 3), name= 'conv2_2'))
base_model.add(Activation('relu', name='relu2_2'))
base_model.add(MaxPooling2D((2,2), strides=(2,2), name='pool2'))

base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(256, (3, 3), name= 'conv3_1'))
base_model.add(Activation('relu', name='relu3_1'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(256, (3, 3), name= 'conv3_2'))
base_model.add(Activation('relu', name='relu3_2'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(256, (3, 3), name= 'conv3_3'))
base_model.add(Activation('relu', name='relu3_3'))
base_model.add(MaxPooling2D((2,2), strides=(2,2), name='pool3'))

base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv4_1'))
base_model.add(Activation('relu', name='relu4_1'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv4_2'))
base_model.add(Activation('relu', name='relu4_2'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv4_3'))
base_model.add(Activation('relu', name='relu4_3'))
base_model.add(MaxPooling2D((2,2), strides=(2,2), name='pool4'))

base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv5_1'))
base_model.add(Activation('relu', name='relu5_1'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv5_2'))
base_model.add(Activation('relu', name='relu5_2'))
base_model.add(ZeroPadding2D((1,1)))
base_model.add(Convolution2D(512, (3, 3), name= 'conv5_3'))
base_model.add(Activation('relu', name='relu5_3'))
base_model.add(MaxPooling2D((2,2), strides=(2,2), name='pool5'))
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Convolution2D(512, (7, 7), name= 'fc6'))
model.add(Activation('relu', name='relu6'))
model.add(Dropout(0.5, name='dropout6'))
model.add(Convolution2D(512, (1, 1), name= 'fc7'))
model.add(Activation('relu', name='relu7'))
model.add(Dropout(0.5, name='dropout7'))
# model.add(Convolution2D(2622, (1, 1), name= 'fc8'))
model.add(Convolution2D(1, (1, 1), name= 'fc8'))
# model.add(Activation('relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Activation('sigmoid', name= 'sigmoid'))

In [14]:
for i in range(num_of_ref_model_layers):
    ref_model_layer = ref_model_layers[i][0,0].name[0]
    if ref_model_layer in base_model_layer_names:
        #we just need to set convolution and fully connected weights
        if ref_model_layer.find("conv") == 0:
            print(i,". ",ref_model_layer)
            base_model_index = basemodel_layer_names.index(ref_model_layer)
            weights = ref_model_layers[i][0,0].weights[0,0]
            bias = ref_model_layers[i][0,0].weights[0,1]
            base_model.layers[base_model_index].set_weights([weights, bias[:,0]])
        elif ref_model_layer.find("fc") == 0:
            print(i,". ",ref_model_layer)
            try:
              model_index = model_layer_names.index(ref_model_layer)+1
              weights = ref_model_layers[i][0,0].weights[0,0]
              bias = ref_model_layers[i][0,0].weights[0,1]
              model.layers[model_index].set_weights([weights, bias[:,0]])
            except: pass

0 .  conv1_1
2 .  conv1_2
5 .  conv2_1
7 .  conv2_2
10 .  conv3_1
12 .  conv3_2
14 .  conv3_3
17 .  conv4_1
19 .  conv4_2
21 .  conv4_3
24 .  conv5_1
26 .  conv5_2
28 .  conv5_3
31 .  fc6
34 .  fc7
37 .  fc8


In [15]:
# model.summary()

In [16]:
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.applications.vgg16 import preprocess_input
# image = load_img('./data/train/F/2314_1956-07-30_2014.jpg', target_size=(224, 224))
# image = img_to_array(image)
# image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# image = preprocess_input(image)/225

In [17]:
# model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
# model.predict(image)

In [18]:
# model.fit(x = image, y = [0], epochs = 3)

In [19]:
# model.predict(image)

# Import pictures data

In [20]:
# !mkdir ./data/train/
# !mkdir ./data/valid/
# !mkdir ./data/test/
# !mkdir ./data/train/M
# !mkdir ./data/train/F
# !mkdir ./data/valid/M
# !mkdir ./data/valid/F
# !mkdir ./data/test/M
# !mkdir ./data/test/F
# !mv -v ./combined/aligned/*F/* ./data/train/F/
# !mv -v ./combined/aligned/*M/* ./data/train/M/
# !ls ./data/train/F | wc -l 
# !ls ./data/train/M | wc -l 
''' Take 10% into valid, 10% into test '''
# os.chdir("./data/train/F")
# !ls | shuf -n 1431 | xargs -i mv {} ../../valid/F
# !ls | shuf -n 1431 | xargs -i mv {} ../../test/F
# os.chdir("../../../")
# os.chdir("./data/train/M")
# !ls | shuf -n 1512 | xargs -i mv {} ../../valid/M
# !ls | shuf -n 1512 | xargs -i mv {} ../../test/M
# os.chdir("../../../")

!ls ./data/train/F | wc -l 
!ls ./data/train/M | wc -l 
!ls ./data/valid/F | wc -l 
!ls ./data/valid/M | wc -l 
!ls ./data/test/F | wc -l 
!ls ./data/test/M | wc -l 

# !mv -v ./data/test/M/* ./data/train/M/
# !mv -v ./data/test/F/* ./data/train/F/
# !mv -v ./data/valid/M/* ./data/train/M/
# !mv -v ./data/valid/F/* ./data/train/F/

11450
12101
1431
1512
1431
1512


In [21]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [65]:
# example of progressively loading images from file

# create generator
datagen = ImageDataGenerator(rescale=1./255)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('data/train/', target_size=(224, 224), batch_size=64, class_mode='binary', shuffle=True)
val_it = datagen.flow_from_directory('data/valid/', target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=True)
test_it = datagen.flow_from_directory('data/test/', target_size=(224, 224), batch_size=1, class_mode='binary', shuffle=False)
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))


Found 23551 images belonging to 2 classes.
Found 2943 images belonging to 2 classes.
Found 2943 images belonging to 2 classes.
Batch shape=(64, 224, 224, 3), min=0.000, max=1.000


# Train and Fine-tune 

In [ ]:
# model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
import keras
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save("./model{}.hd5".format(epoch))
saver = CustomSaver()

In [ ]:
model.fit_generator(train_it, 
                    steps_per_epoch=367, 
                    validation_data=val_it, 
                    validation_steps=5, 
                    callbacks=[saver], 
                    epochs = 3)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
367/367 [==============================] - ETA: 0s - loss: 0.3415 - accuracy: 0.9411 WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./model0.hd5/assets
367/367 [==============================] - 12494s 34s/step - loss: 0.3415 - accuracy: 0.9411 - val_loss: 0.3143 - val_accuracy: 0.9375
Epoch 2/3
367/367 [==============================] - 12459s 34s/step - loss: 0.2549 - accuracy: 0.9664 - val_loss: 0.2350 - val_accuracy: 0.9625
Epoch 3/3
367/367 [===

In [23]:
model = load_model("./model2.hd5")

In [70]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(test_it, 2943)
y_pred = [0 if i < 0.5 else 1 for i in Y_pred]

In [26]:
unique, counts = np.unique(test_it.classes, return_counts=True)
dict(zip(unique, counts))

{0: 1431, 1: 1512}

In [71]:
print('Confusion Matrix')
print(confusion_matrix(test_it.classes, y_pred))
target_names = list(test_it.class_indices.keys())   
print(classification_report(test_it.classes, y_pred, target_names=target_names))

Confusion Matrix
[[1388   43]
 [  62 1450]]
              precision    recall  f1-score   support

      Female       0.96      0.97      0.96      1431
        Male       0.97      0.96      0.97      1512

    accuracy                           0.96      2943
   macro avg       0.96      0.96      0.96      2943
weighted avg       0.96      0.96      0.96      2943



In [72]:
tn, fp, fn, tp = confusion_matrix(test_it.classes, y_pred).ravel()
print(tn, fp, fn, tp)

1388 43 62 1450


In [30]:
model.evaluate_generator(test_it, 92)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.1511135995388031, 0.965624988079071]

In [ ]:
# steps_per_epoch * batch_size = number_of_rows_in_train_data
# https://sefiks.com/2019/07/15/how-to-convert-matlab-models-to-keras/
# https://keras.io/api/preprocessing/image/#imagedatagenerator-class